# Hurricane Fiona (2022-09-16/22) around Puerto Rico

In [ ]:
from pathlib import Path

path = Path("data", "goes16-hurricane")
path.mkdir(parents=True, exist_ok=True)

In [ ]:
min_lon, max_lon, min_lat, max_lat = bbox = -72, -61, 15, 21

min_time = "2022-09-16T12:00:00Z"
max_time = "2022-09-22T00:00:00Z"

In [ ]:
from datetime import datetime

from goes2go import GOES

blend = "ABI-L2-MCMIPF"

g = GOES(satellite=16, product=blend, domain="C")
df_goes = g.df(
    start=datetime.strptime(min_time, "%Y-%m-%dT%H:%M:%SZ"),
    end=datetime.strptime(max_time, "%Y-%m-%dT%H:%M:%SZ"),
)

df_goes

In [ ]:
import matplotlib.ticker as mticker
import xarray as xr


def make_frame():
    fig, ax = plt.subplots(
        figsize=(15, 12), subplot_kw={"projection": ccrs.PlateCarree()}
    )
    ax.set_extent(bbox, ccrs.PlateCarree())
    ax.coastlines(color="white", linewidth=0.75)
    gl = ax.gridlines(draw_labels=True, linewidth=0.5)
    gl.xpadding = gl.ypadding = 5
    gl.xlocator = mticker.FixedLocator(np.arange(bbox[0], bbox[1], 2))
    gl.ylocator = mticker.FixedLocator(np.arange(bbox[2], bbox[3], 2))
    gl.right_labels = gl.top_labels = False
    gl.xlabel_style = gl.ylabel_style = {"fontsize": 16, "rotation": 0}
    return fig, ax


def read_rgb(fname_from_df):
    fname = fname_from_df.split("/")[-1]
    fname = path.joinpath(f"rgb-{fname}")
    return xr.open_dataarray(fname)


def round_datetime(ds):
    round_secs = str(ds["t"].to_numpy()).split(".")[0]
    return f"{round_secs[:-2]}00"

In [ ]:
datetimes = []
images = {}


for k, fname in enumerate(df_goes["file"]):
    ds = read_rgb(fname)
    datetimes.append(round_datetime(ds))
    images.update({k: ds})

In [ ]:
import pandas as pd

new_index = pd.DatetimeIndex(pd.to_datetime(pd.Series(datetimes)))
new_index = new_index.tz_localize("UTC")

## Download Saildrone

In [ ]:
import pandas as pd
from erddapy import ERDDAP

e = ERDDAP(
    server="https://data.pmel.noaa.gov/generic/erddap",
    protocol="tabledap",
)

kw = {
    "min_lon": min_lon,
    "max_lon": max_lon,
    "min_lat": min_lat,
    "max_lat": max_lat,
    "min_time": min_time,
    "max_time": max_time,
    # This is a hack and not a reliable metadata to identify Saildrones.
    "institution": "Saildrone",
}


search_url = e.get_search_url(response="csv", **kw)
search = pd.read_csv(search_url)
dataset_ids = search["Dataset ID"].values

saildrones_list = "\n".join(dataset_ids)
print(f"Found {len(dataset_ids)} Saildrones Datasets.")

In [ ]:
def request_track(dataset_id, auv):
    fname = path.joinpath(f"{auv}-{dataset_id}.csv")
    if not fname.exists():
        e.constraints = {
            "time>=": min_time,
            "time<=": max_time,
            "latitude>=": min_lat,
            "latitude<=": max_lat,
            "longitude>=": min_lon,
            "longitude<=": max_lon,
        }
        e.protocol = "tabledap"
        e.variables = ["time", "longitude", "latitude"]
        e.dataset_id = dataset_id
        # Drop units in the first line and NaNs.
        df = e.to_pandas(response="csv", skiprows=(1,)).dropna()
        df["time"] = pd.to_datetime(df["time"])
        df = df.set_index("time")
        df.to_csv(fname)
    else:
        df = pd.read_csv(fname, index_col="time", parse_dates=True)
    return df

In [ ]:
from requests.exceptions import HTTPError

saildrones = {}
for dataset_id in dataset_ids:
    try:
        df = request_track(dataset_id, auv="saildrone")
    except HTTPError:
        print(f"Could not download {dataset_id}")
        continue
    saildrones.update({dataset_id: df})

## Download Glider

In [ ]:
e = ERDDAP(
    server="https://gliders.ioos.us/erddap",
    protocol="tabledap",
)

kw = {
    "min_lon": min_lon,
    "max_lon": max_lon,
    "min_lat": min_lat,
    "max_lat": max_lat,
    "min_time": min_time,
    "max_time": max_time,
    "cdm_data_type": "trajectoryprofile",
}


search_url = e.get_search_url(response="csv", **kw)
search = pd.read_csv(search_url)
gliders = search["Dataset ID"].values
# filter out delayed
dataset_ids = [glider for glider in gliders if "delayed" not in glider]

gliders_list = "\n".join(dataset_ids)
print(f"Found {len(dataset_ids)} Glider Datasets.")

In [ ]:
gliders = {}
for dataset_id in dataset_ids:
    try:
        df = request_track(dataset_id, auv="glider")
    except HTTPError:
        print(f"Could not download {dataset_id}")
        continue
    gliders.update({dataset_id: df})

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap


def make_map(figsize=(12, 12), projection=ccrs.PlateCarree()):
    fig, ax = plt.subplots(
        subplot_kw={"projection": projection},
        figsize=figsize,
    )
    ax.set_extent(bbox)
    ax.coastlines()
    return fig, ax


def make_colors(hex_color):
    hex_color = hex_color.strip("#")
    rgb = tuple(int(hex_color[k : k + 2], 16) for k in (0, 2, 4))
    mpl = np.array(rgb) / 255
    alphas = [
        [mpl[0], mpl[1], mpl[2], 0.0],
        [mpl[0], mpl[1], mpl[2], 0.5],
        [mpl[0], mpl[1], mpl[2], 1.0],
    ]
    return LinearSegmentedColormap.from_list("", alphas)

## Download Argo

In [ ]:
fname = path.joinpath("argo.csv")


if not fname.exists():
    from argopy import DataFetcher as ArgoDataFetcher

    fetcher = ArgoDataFetcher().region(
        [min_lon, max_lon, min_lat, max_lat, 0, 1.5, min_time, max_time]
    )

    argo = fetcher.to_dataframe()
    argo = argo.drop_duplicates(subset=["PLATFORM_NUMBER"])
    argo = argo[["TIME", "LONGITUDE", "LATITUDE"]].set_index("TIME")
    argo.index = argo.index.tz_localize("UTC")
    argo.to_csv(fname)
else:
    argo = pd.read_csv(fname, index_col="TIME", parse_dates=True)

In [ ]:
colors = ["#ff7f50", "#c20078", "#ffcc33"]


def reindex_and_interpolate(df, new_index):
    return (
        df.reindex(df.index.union(new_index))
        .interpolate(method="pad", limit_direction="forward", limit=1)
        .loc[new_index]
    )


inter_gliders = {}
for glider, df in gliders.items():
    # Some delayed gliders have duplicated index.
    df = df.drop_duplicates()
    df = reindex_and_interpolate(df, new_index)
    inter_gliders.update({glider: df})


inter_sdrones = {}
for saildrone, df in saildrones.items():
    df = df.drop_duplicates()
    df = reindex_and_interpolate(df, new_index)
    inter_sdrones.update({saildrone: df})


inter_argo = reindex_and_interpolate(argo, new_index)

### Check interpolation

In [ ]:
fig, ax = make_map()

for glider, df in gliders.items():
    df = df.dropna()
    ax.plot(
        df["longitude"], df["latitude"], color=colors[0], linestyle="none", marker="*"
    )


for saildrone, df in saildrones.items():
    df = df.dropna()
    ax.plot(
        df["longitude"], df["latitude"], color=colors[1], linestyle="none", marker="o"
    )


ax.plot(
    argo["LONGITUDE"], argo["LATITUDE"], color=colors[2], linestyle="none", marker="d"
);

In [ ]:
fig, ax = make_map()

for glider, df in inter_gliders.items():
    ax.plot(
        df["longitude"], df["latitude"], color=colors[0], linestyle="none", marker="*"
    )


for saildrone, df in inter_sdrones.items():
    ax.plot(
        df["longitude"], df["latitude"], color=colors[1], linestyle="none", marker="o"
    )


ax.plot(
    inter_argo["LONGITUDE"],
    inter_argo["LATITUDE"],
    color=colors[2],
    linestyle="none",
    marker="d",
);

## Animation

In [ ]:
import gc

from goes2go.data import _as_xarray_MP

from utils import smaller_image


# Init
def init():
    ax.set_xlim(min_lon, max_lon)
    ax.set_ylim(min_lat, max_lat)
    return (title,)


ds = _as_xarray_MP(df_goes.iloc[0].file, save_dir=path, i=1, n=1, verbose=True)
ds, imshow_kw = smaller_image(ds, min_lon, max_lon, min_lat, max_lat)

fig, ax = make_frame()
im = ax.imshow(ds, **imshow_kw)
title = ax.text(
    0.5,
    1.05,
    "",
    bbox={"facecolor": "w", "alpha": 0.5, "pad": 5},
    transform=ax.transAxes,
    ha="center",
    fontsize=24,
)

# Glider
intensity_glider = []
x_glider, y_glider = [], []
cmap_glider = make_colors(colors[0])
scatter_glider = ax.scatter([], [], s=30, c=[], cmap=cmap_glider, vmin=0, vmax=1)


def update_glider(k):
    global scatter_glider, intensity_glider, x_glider, y_glider
    x, y = [], []
    for glider, df in inter_gliders.items():
        point = df.iloc[k]
        x.append(point["longitude"])
        y.append(point["latitude"])
    x_glider.extend(x)
    y_glider.extend(y)
    time_date = df.index[k]
    arr = np.c_[x_glider, y_glider]
    scatter_glider.set_offsets(arr)
    intensity_glider = np.concatenate(
        (np.array(intensity_glider) * 0.97, np.ones(len(x)))
    )
    scatter_glider.set_array(intensity_glider)
    return scatter_glider, time_date


# Saildrone
intensity_sdrone = []
x_sdrone, y_sdrone = [], []
cmap_sdrone = make_colors(colors[1])
scatter_sdrone = ax.scatter(
    [], [], s=30, c=[], marker="^", cmap=cmap_sdrone, vmin=0, vmax=1
)


def update_sdrone(k):
    global scatter_sdrone, intensity_sdrone, x_sdrone, y_sdrone
    x, y = [], []
    for sdrone, df in inter_sdrones.items():
        point = df.iloc[k]
        x.append(point["longitude"])
        y.append(point["latitude"])
    x_sdrone.extend(x)
    y_sdrone.extend(y)
    time_date = df.index[k]
    arr = np.c_[x_sdrone, y_sdrone]
    scatter_sdrone.set_offsets(arr)
    intensity_sdrone = np.concatenate(
        (np.array(intensity_sdrone) * 0.92, np.ones(len(x)))
    )
    scatter_sdrone.set_array(intensity_sdrone)
    return scatter_sdrone, time_date


# Argo
intensity_argo = []
x_argo, y_argo = [], []
cmap_argo = make_colors(colors[2])
scatter_argo = ax.scatter(
    x_argo, y_argo, s=40, c=[], marker="d", cmap=cmap_argo, vmin=0, vmax=1
)


def update_argo(k):
    global scatter_argo, intensity_argo, x_argo, y_argo
    x, y = [], []
    point = inter_argo.iloc[k]
    x.append(point["LONGITUDE"])
    y.append(point["LATITUDE"])
    x_argo.extend(x)
    y_argo.extend(y)
    arr = np.c_[x_argo, y_argo]
    scatter_argo.set_offsets(arr)
    intensity_argo = np.concatenate((np.array(intensity_argo) * 0.96, np.ones(len(x))))
    scatter_argo.set_array(intensity_argo)
    return (scatter_argo,)


# GOES
def update_goes(k):
    global im
    ds = images[k]
    im = ax.imshow(ds, **imshow_kw, animated=True)
    # This is way more efficient then calling imshow but for some reason the imshow kw are not preserved.
    # im.set_array(ds["rgb"])
    ds.close()
    del ds
    return im


def update(k):
    _, time_date = update_glider(k)
    update_sdrone(k)
    update_argo(k)
    update_goes(k)
    title.set_text(f"{new_index[k].tz_localize(None)}")
    gc.collect()
    fname = f"frame_{str(k).zfill(3)}.png"
    print(fname)
    plt.savefig(fname)
    return (title,)


for k, idx in enumerate(new_index):
    update(k)

In [ ]:
import subprocess

fps = 15

subprocess.call(
    f'ffmpeg -framerate {fps} -pattern_type glob -i "*.png" -c:v libx264 -pix_fmt yuv420p out_{fps}.mp4',
    shell=True,
)
subprocess.call(
    f'ffmpeg -y -i out_{fps}.mp4 -filter:v "crop=in_w-2*100:in_h-2*200" cropped_out_{fps}.mp4',
    shell=True,
)